## Colab-related

In [ ]:
# import data into google colab
from google.colab import files
import io
uploaded = files.upload()

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# unzip data
import zipfile
zf = zipfile.ZipFile(io.BytesIO(uploaded['CrossSections.zip']), "r")
zf.extractall('CrossSections')

## Specifications


In [2]:
# Standard import statements
#
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras # type: ignore
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard # type: ignore

# Import local modules
#
from sdata import SData
from tfimage import TFImage
from cvimage import CVImage, NuclearLayer
from sdata import DataGenerator as DGen

In [3]:
# folders and files
#
sys_folder   = '/Volumes/X2/Projects/staging/Data/'
drive_folder = '/Volumes/X2/Projects/staging/Data/'
data_folder  = sys_folder + 'data/'

In [4]:
# training parameters
#  
image_length = 128
shufflenum   = 100
nBatchTrain  = 25
nBatchVal    = 25
AUTOTUNE     = tf.data.AUTOTUNE

# define train, val, test data
#
test = [6,7]
val  = [8,9]
ignore = [26, 31, 20, 21, 22, 23, 24, 25, 27]
model_name = f"staging_nl_test_{'-'.join(map(str, test))}_val_{'-'.join(map(str, val))}_ignore_{'-'.join(map(str, ignore))}"
print(f"Model name: {model_name}")

Model name: staging_nl_test_6-7_val_8-9_ignore_26-31-20-21-22-23-24-25-27


## Data preparation

In [5]:
# create data generator and test it
#
size        = (512, 512)
padding     = 44
npoints     = 60
inward      = 40
outward     = -24
length      = 128
d = DGen(data_folder, test, val, ignore, size, padding, npoints, inward, outward, length)
image, id = next(d('test'))
print(f"Image shape: {image.shape}") # type: ignore
print(f"Image id: {id}")
print(f"Image dtype: {image.dtype}") # type: ignore
print(f"Id dtype: {id.dtype}")

yes_shuffle: True
list_type: test
Image shape: (64, 128, 1)
Image id: 0.4213887718923566
Image dtype: <dtype: 'float32'>
Id dtype: float64


2024-08-25 14:01:08.052556: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-08-25 14:01:08.052592: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-25 14:01:08.052603: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-25 14:01:08.052628: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-25 14:01:08.052648: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# specify tf dataset using data generator and print a few data
#
output_signature = (
  tf.TensorSpec(shape=(None, None, 1), dtype=tf.float32),   # type: ignore
  tf.TensorSpec(shape=(), dtype=tf.float64))                # type: ignore

train_ds = tf.data.Dataset.from_generator(
  lambda: d('test'), 
  output_signature=output_signature
)
# train_ds = train_ds.shuffle(shufflenum).batch(nBatchTrain).prefetch(AUTOTUNE)

print(train_ds)

<_FlatMapDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>


In [ ]:
i = 0
try:
  for I, id in train_ds.take(10): # type: ignore
    # plot in figure that is 512 by 512
    plt.figure(figsize=(20, 20))
    print(f"i = {i}")
    print(f"Image shape: {I.shape}")
    print(f"Image id: {id}")
    plt.imshow(I[:, :, 0], cmap='gray')
    plt.show()
    plt.close()
    i += 1
except tf.errors.OutOfRangeError:
  print("Reached the end of the dataset.")

## Model

In [12]:
# define the CNN model
#
def create_cnn_model(input_shape):
    model = keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)  # Assuming regression task, adjust if classification
    ])
    return model

# Create the model
image_shape = image.shape # type: ignore
model = create_cnn_model(image_shape)

In [13]:
# compile the model
model.compile(optimizer='adam',
              loss='mse',       # Mean Squared Error for regression
              metrics=['mae'])  # Mean Absolute Error
# print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 128, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,097,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,153,025 (8.21 MB)

 Trainable params: 2,153,025 (8.21 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# train the model
model.fit(train_ds, epochs=10)

Epoch 1/10


ValueError: Exception encountered when calling MaxPooling2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1/max_pooling2d_1/MaxPool2d}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](sequential_1/conv2d_1/Relu)' with input shapes: [?,?,1,32].[0m

Arguments received by MaxPooling2D.call():
  • inputs=tf.Tensor(shape=(None, None, 1, 32), dtype=float32)